In [36]:
import pandas as pd
import psycopg2 as pg

# Import SQL Alchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey



In [37]:

departments_path = pd.read_csv('../data/departments.csv')
dept_emp = pd.read_csv('../data/dept_emp.csv')
dept_manager = pd.read_csv('../data/dept_manager.csv')
employees = pd.read_csv('../data/employees.csv')
salaries = pd.read_csv('../data/salaries.csv')
titles = pd.read_csv('../data/titles.csv')


In [38]:
# Create classes with primary keys and foreign keys

class Department (Base):
    __tablename__ = 'departments'
    __table_args__ = {'extend_existing': True}
    dept_no = Column(String, primary_key = True)
    dept_name = Column(String)
    
class Title (Base):
    __tablename__ = 'titles'
    __table_args__ = {'extend_existing': True}
    title_id = Column(String, primary_key = True)
    title = Column(String)
    
class Employee(Base):
    __tablename__ = 'employees'
    __table_args__ = {'extend_existing': True}
    emp_no = Column(Integer, primary_key = True)
    emp_title_id = Column(String, ForeignKey('titles.title_id'))
    birth_date = Column(Date)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)
    hire_date = Column(Date)
    #title = relationship('Title', back_populates = 'employees')
    title_employees = relationship('Title', foreign_keys = 'Employee.emp_title_id')
#Title.employees = relationship('Employee', order_by = Employee.emp_title_id, back_populates = 'title')

class Dept_employee (Base):
    __tablename__ = 'dept_emp'
    __table_args__ = {'extend_existing': True}
    emp_no = Column(Integer, ForeignKey('employees.emp_no'),primary_key = True)
    dept_no = Column(Integer, ForeignKey('departments.dept_no'), primary_key = True)
    
    emp_no_deptemp = relationship('Employee', foreign_keys='Dept_employee.emp_no')
    dept_no_deptemp = relationship('Department', foreign_keys='Dept_employee.dept_no')
            
class Salary (Base):
    __tablename__ = 'salaries'
    __table_args__ = {'extend_existing': True}
    emp_no = Column(Integer, ForeignKey('employees.emp_no'),primary_key = True)
    salary = Column(Integer)
    
    emp_no_salary = relationship('Employee', foreign_keys='Salary.emp_no')

In [39]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///Employee.db')

In [40]:
Base.metadata.tables

immutabledict({'departments': Table('departments', MetaData(bind=None), Column('dept_no', String(), table=<departments>, primary_key=True, nullable=False), Column('dept_name', String(), table=<departments>), schema=None), 'titles': Table('titles', MetaData(bind=None), Column('title_id', String(), table=<titles>, primary_key=True, nullable=False), Column('title', String(), table=<titles>), schema=None), 'employees': Table('employees', MetaData(bind=None), Column('emp_no', Integer(), table=<employees>, primary_key=True, nullable=False), Column('emp_title_id', String(), ForeignKey('titles.title_id'), table=<employees>), Column('birth_date', Date(), table=<employees>), Column('first_name', String(), table=<employees>), Column('last_name', String(), table=<employees>), Column('sex', String(), table=<employees>), Column('hire_date', Date(), table=<employees>), schema=None), 'dept_emp': Table('dept_emp', MetaData(bind=None), Column('emp_no', Integer(), ForeignKey('employees.emp_no'), table=<d

In [41]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)